# Training with RLlib PPO
[RLlib](https://docs.ray.io/en/latest/rllib/index.html) is a high-performance, distributed
reinforcement learning library. It is preferable to other RL libraries (e.g. Stable Baselines
3) for `bsk_rl` environments because it steps environments copies asynchronously; because 
of the variable step lengths, variable episode step counts, and long episode reset times, 
stepping each environment independently can increase step throughput by 2-5 times.

<div class="alert alert-warning">

**Warning:** RLlib had a bug that results in an undesirable timeout which stops
training. It has since been resolved: https://github.com/ray-project/ray/pull/45147

</div>


## Define the Environment
A nadir-scanning environment is created, to the one used in [this paper](https://hanspeterschaub.info/Papers/Stephenson2024.pdf). 
The satellite has to collect data while managing the data buffer level and battery level.

First, the satellite class is defined. A custom dynamics model is created that defines
a few additional properties to use in the state.

In [1]:
import numpy as np
from bsk_rl import act, data, obs, sats, scene
from bsk_rl.sim import dyn, fsw

class ScanningDownlinkDynModel(dyn.ContinuousImagingDynModel, dyn.GroundStationDynModel):
    # Define some custom properties to be accessed in the state
    @property
    def instrument_pointing_error(self) -> float:
        r_BN_P_unit = self.r_BN_P/np.linalg.norm(self.r_BN_P) 
        c_hat_P = self.satellite.fsw.c_hat_P
        return np.arccos(np.dot(-r_BN_P_unit, c_hat_P))
    
    @property
    def solar_pointing_error(self) -> float:
        a = self.world.gravFactory.spiceObject.planetStateOutMsgs[
            self.world.sun_index
        ].read().PositionVector
        a_hat_N = a / np.linalg.norm(a)
        nHat_B = self.satellite.sat_args["nHat_B"]
        NB = np.transpose(self.BN)
        nHat_N = NB @ nHat_B
        return np.arccos(np.dot(nHat_N, a_hat_N))

class ScanningSatellite(sats.AccessSatellite):
    observation_spec = [
        obs.SatProperties(
            dict(prop="storage_level_fraction"),
            dict(prop="battery_charge_fraction"),
            dict(prop="wheel_speeds_fraction"),
            dict(prop="instrument_pointing_error", norm=np.pi),
            dict(prop="solar_pointing_error", norm=np.pi)
        ),
        obs.OpportunityProperties(
            dict(prop="opportunity_open", norm=5700),
            dict(prop="opportunity_close", norm=5700),
            type="ground_station",
            n_ahead_observe=1,
        ),
        obs.Eclipse(norm=5700),
        obs.Time(),
    ]
    action_spec = [
        act.Scan(duration=180.0),
        act.Charge(duration=120.0),
        act.Downlink(duration=60.0),
        act.Desat(duration=60.0),
    ]
    dyn_type = ScanningDownlinkDynModel
    fsw_type = fsw.ContinuousImagingFSWModel

Next, parameters are set. Since this scenario is focused on maintaining acceptable data
and power levels, these are tuned to create a sufficiently interesting mission.

In [2]:
sat = ScanningSatellite(
    "Scanner-1",
    sat_args=dict(
        # Data
        dataStorageCapacity=5000 * 8e6,  # bits
        storageInit=lambda: np.random.uniform(0.0, 0.8) * 5000 * 8e6,
        instrumentBaudRate=0.5 * 8e6,
        transmitterBaudRate=-50 * 8e6,
        # Power
        batteryStorageCapacity=200 * 3600,  # W*s
        storedCharge_Init=lambda: np.random.uniform(0.3, 1.0) * 200 * 3600,
        basePowerDraw=-10.0,  # W
        instrumentPowerDraw=-30.0,  # W
        transmitterPowerDraw=-25.0,  # W
        thrusterPowerDraw=-80.0,  # W
        panelArea=0.25,
        # Attitude
        imageAttErrorRequirement=0.1,
        imageRateErrorRequirement=0.1,
        disturbance_vector=lambda: np.random.normal(scale=0.0001, size=3),  # N*m
        maxWheelSpeed=6000.0,  # RPM
        wheelSpeeds=lambda: np.random.uniform(-3000, 3000, 3),
        desatAttitude="nadir",
    )
)

Finally, the environment arguments are set. Stepping through this environment is 
demonstrated at the bottom of the page.

In [3]:
duration = 5 * 5700.0  # About 5 orbits
env_args = dict(
    satellite=sat,
    scenario=scene.UniformNadirScanning(value_per_second=1/duration),
    rewarder=data.ScanningTimeReward(),
    time_limit=duration,
    failure_penalty=-1.0,
    terminate_on_time_limit=True,
)

## Configure Ray and PPO

The `bsk_rl` package supplies a utility to make logging information at the end of episodes
easier. This is useful to see how an agent's policy is changing over time, using a
monitoring program such as [TensorBoard](https://www.tensorflow.org/tensorboard).

In [4]:
from bsk_rl.utils.rllib import EpisodeDataCallbacks

class CustomDataCallbacks(EpisodeDataCallbacks):
    def pull_env_metrics(self, env):
        reward = env.rewarder.cum_reward
        reward = sum(reward.values()) / len(reward)
        orbits = env.simulator.sim_time / (95 * 60)

        data = dict(
            reward=reward,
            # Are satellites dying, and how and when?
            alive=float(env.satellite.is_alive()),
            rw_status_valid=float(env.satellite.dynamics.rw_speeds_valid()),
            battery_status_valid=float(env.satellite.dynamics.battery_valid()),
            orbits_complete=orbits,
        )
        if orbits > 0:
            data["reward_per_orbit"] = reward / orbits
        if not env.satellite.is_alive():
            data["orbits_complete_partial_only"] = orbits
            
        return  data

Then, PPO (or some other algorithm) can be configured. Of particular importance
are setting `sample_timeout_s` and `metrics_episode_collection_timeout_s` to appropriately
high values for this environment.

In [5]:
from bsk_rl import SatelliteTasking
from bsk_rl.utils.rllib import unpack_config
from ray.rllib.algorithms.ppo import PPOConfig

N_CPUS = 3

training_args = dict(
    lr=0.00003,
    gamma=0.999,
    train_batch_size=250,  # usually a larger number, like 2500
    num_sgd_iter=10,
    model=dict(fcnet_hiddens=[512, 512], vf_share_layers=False),
    lambda_=0.95,
    use_kl_loss=False,
    clip_param=0.1,
    grad_clip=0.5,
)

config = (
    PPOConfig()
    .training(**training_args)
    .env_runners(num_env_runners=N_CPUS-1, sample_timeout_s=1000.0)
    .environment(
        env=unpack_config(SatelliteTasking),
        env_config=env_args,
    )
    .callbacks(CustomDataCallbacks)
    .reporting(
        metrics_num_episodes_for_smoothing=1,
        metrics_episode_collection_timeout_s=180,
    )
    .checkpointing(export_native_model_files=True)
    .framework(framework="tf2")
    # Uncomment these lines if using the new RLlib API stack
    # .framework(framework="torch")
    # .api_stack(
    #     enable_rl_module_and_learner=True,
    #     enable_env_runner_and_connector_v2=True,
    # )

)

Once the PPO configuration has been set, `ray` can be started and the agent can be
trained.

Training on a reasonably modern machine, we can achieve 5M steps over 32 processors in 6
to 18 hours, depending on specific environment configurations.

In [6]:
import ray
from ray import tune

ray.init(
    ignore_reinit_error=True,
    num_cpus=N_CPUS,
    object_store_memory=2_000_000_000,  # 2 GB
)

# Run the training
tune.run(
    "PPO",
    config=config.to_dict(),
    stop={"training_iteration": 10},  # Adjust the number of iterations as needed
    checkpoint_freq=10,
    checkpoint_at_end=True
)

# Shutdown Ray
ray.shutdown()

2024-09-11 09:26:42,978	INFO worker.py:1783 -- Started a local Ray instance.


2024-09-11 09:26:43,273	INFO tune.py:616 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


/Users/markstephenson/avslab/refactor/.venv_refactor/lib/python3.10/site-packages/gymnasium/spaces/box.py:130: UserWarning: WARN: Box bound precision lowered by casting to float32
  gym.logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
/Users/markstephenson/avslab/refactor/.venv_refactor/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:164: UserWarning: WARN: The obs returned by the `reset()` method was expecting numpy array dtype to be float32, actual type: float64
  logger.warn(
/Users/markstephenson/avslab/refactor/.venv_refactor/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:188: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


(PPO pid=63099) Install gputil for GPU system monitoring.


(RolloutWorker pid=63101) 2024-09-11 09:26:53,433 sats.satellite.Scanner-1       WARNING    <7500.00> Scanner-1: failed battery_valid check
(RolloutWorker pid=63101) WARNING:bsk_rl.sats.satellite.Scanner-1:failed battery_valid check


...

Trial name,agent_timesteps_total,counters,custom_metrics,env_runners,episode_media,fps,info,num_agent_steps_sampled,num_agent_steps_sampled_lifetime,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_lifetime,num_env_steps_sampled_this_iter,num_env_steps_sampled_throughput_per_sec,num_env_steps_trained,num_env_steps_trained_this_iter,num_env_steps_trained_throughput_per_sec,num_healthy_workers,num_in_flight_async_sample_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,timers
PPO_SatelliteTasking_Unpacked_3fde2_00000,2500,"{'num_env_steps_sampled': 2500, 'num_env_steps_trained': 2500, 'num_agent_steps_sampled': 2500, 'num_agent_steps_trained': 2500}",{},"{'episode_reward_max': 0.4164912280701755, 'episode_reward_min': 0.4164912280701755, 'episode_reward_mean': 0.4164912280701755, 'episode_len_mean': 245.0, 'episode_media': {}, 'episodes_timesteps_total': 245, 'policy_reward_min': {'default_policy': 0.4164912280701755}, 'policy_reward_max': {'default_policy': 0.4164912280701755}, 'policy_reward_mean': {'default_policy': 0.4164912280701755}, 'custom_metrics': {'reward_mean': 0.4164912280701755, 'reward_min': 0.4164912280701755, 'reward_max': 0.4164912280701755, 'alive_mean': 1.0, 'alive_min': 1.0, 'alive_max': 1.0, 'rw_status_valid_mean': 1.0, 'rw_status_valid_min': 1.0, 'rw_status_valid_max': 1.0, 'battery_status_valid_mean': 1.0, 'battery_status_valid_min': 1.0, 'battery_status_valid_max': 1.0, 'orbits_complete_mean': 5.0, 'orbits_complete_min': 5.0, 'orbits_complete_max': 5.0, 'reward_per_orbit_mean': 0.08329824561403511, 'reward_per_orbit_min': 0.08329824561403511, 'reward_per_orbit_max': 0.08329824561403511}, 'hist_stats': {'episode_reward': [0.4164912280701755], 'episode_lengths': [245], 'policy_default_policy_reward': [0.4164912280701755]}, 'sampler_perf': {'mean_raw_obs_processing_ms': 1.5414264276445055, 'mean_inference_ms': 0.6536105268007274, 'mean_action_processing_ms': 0.048136730178845205, 'mean_env_wait_ms': 7.8649398901289125, 'mean_env_render_ms': 0.0}, 'num_faulty_episodes': 0, 'connector_metrics': {'ObsPreprocessorConnector_ms': 0.002956390380859375, 'StateBufferConnector_ms': 0.0017404556274414062, 'ViewRequirementAgentConnector_ms': 0.04982948303222656}, 'num_episodes': 1, 'episode_return_max': 0.4164912280701755, 'episode_return_min': 0.4164912280701755, 'episode_return_mean': 0.4164912280701755, 'episodes_this_iter': 1}",{},158.18,"{'learner': {'default_policy': {'learner_stats': {'cur_kl_coeff': 0.0003906250058207661, 'cur_lr': 2.9999999242136255e-05, 'total_loss': -0.0073658274, 'policy_loss': -0.008735873, 'vf_loss': 0.0013696703, 'vf_explained_var': -0.50131404, 'kl': 0.0009595014, 'entropy': 1.36483, 'entropy_coeff': 0.0}, 'custom_metrics': {}, 'num_agent_steps_trained': 125.0, 'num_grad_updates_lifetime': 190.5, 'diff_num_grad_updates_vs_sampler_policy': 9.5}}, 'num_env_steps_sampled': 2500, 'num_env_steps_trained': 2500, 'num_agent_steps_sampled': 2500, 'num_agent_steps_trained': 2500}",2500,2500,2500,2500,2500,250,158.395,2500,250,158.395,2,0,0,250,"{'cpu_util_percent': 21.733333333333334, 'ram_util_percent': 82.0}","{'training_iteration_time_ms': 1541.529, 'restore_workers_time_ms': 0.009, 'training_step_time_ms': 1541.498, 'sample_time_ms': 1414.277, 'learn_time_ms': 123.135, 'learn_throughput': 2030.287, 'synch_weights_time_ms': 3.862}"


(RolloutWorker pid=63100) 2024-09-11 09:27:04,256 sats.satellite.Scanner-1       WARNING    <12780.00> Scanner-1: failed battery_valid check [repeated 3x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)
(RolloutWorker pid=63100) WARNING:bsk_rl.sats.satellite.Scanner-1:failed battery_valid check [repeated 3x across cluster]


(PPO pid=63099) WARNING:tensorflow:Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.
(PPO pid=63099) WARNING:tensorflow:Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.
2024-09-11 09:27:08,339	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/Users/markstephenson/ray_results/PPO_2024-09-11_09-26-43' in 0.0168s.


(PPO pid=63099) /Users/markstephenson/avslab/refactor/.venv_refactor/lib/python3.10/site-packages/keras/src/initializers/__init__.py:144: UserWarning: The `keras.initializers.serialize()` API should only be used for objects of type `keras.initializers.Initializer`. Found an instance of type <class 'function'>, which may lead to improper serialization.
(PPO pid=63099)   warnings.warn(
(PPO pid=63099) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/Users/markstephenson/ray_results/PPO_2024-09-11_09-26-43/PPO_SatelliteTasking_Unpacked_3fde2_00000_0_2024-09-11_09-26-43/checkpoint_000000)


2024-09-11 09:27:08,789	INFO tune.py:1041 -- Total run time: 25.52 seconds (25.02 seconds for the tuning loop).


## Loading the Policy Network

The policy network can be found in the `model` subdirectory of the checkpoint output.

## Stepping Through the Environment

The environment is stepped through with random actions to give a sense of how it acts.

In [7]:
env = SatelliteTasking(**env_args, log_level="INFO")
env.reset()
terminated = False
while not terminated:
    action = env.action_space.sample()
    observation, reward, terminated, truncated, info = env.step(action)

2024-09-11 09:27:10,260 gym                            INFO       Resetting environment with seed=2612025592


2024-09-11 09:27:10,357 sats.satellite.Scanner-1       INFO       <0.00> Scanner-1: Finding opportunity windows from 0.00 to 28500.00 seconds


2024-09-11 09:27:10,414 gym                            INFO       <0.00> Environment reset


2024-09-11 09:27:10,415 gym                            INFO       <0.00> === STARTING STEP ===


2024-09-11 09:27:10,415 sats.satellite.Scanner-1       INFO       <0.00> Scanner-1: action_nadir_scan tasked for 180.0 seconds


...

2024-09-11 09:27:12,877 sats.satellite.Scanner-1       INFO       <28440.00> Scanner-1: setting timed terminal event at 28560.0


2024-09-11 09:27:12,881 data.base                      INFO       <28500.00> Data reward: {'Scanner-1': 0.0}


2024-09-11 09:27:12,882 gym                            INFO       <28500.00> Step reward: 0.0


2024-09-11 09:27:12,882 gym                            INFO       <28500.00> Episode terminated: True


2024-09-11 09:27:12,882 gym                            INFO       <28500.00> Episode truncated: True
